In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

##### 读取数据集

In [2]:
base_dir = r"./data/FourWeather/"

In [3]:
from torchvision import transforms

In [4]:
transform = transforms.Compose([
    transforms.Resize((192,192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

In [5]:
BATCH_SIZE = 8

In [6]:
train_ds = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"train"),
    transform = transform
)

test_ds  = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"test"),
    transform = transform
)

train_dl = torch.utils.data.DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True)
test_dl  = torch.utils.data.DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=True)

In [7]:
class_to_id = train_ds.class_to_idx
id_to_class = dict((v,k) for k,v in train_ds.class_to_idx.items())

##### 创建/使用预训练 模型

###### 预训练模型的使用步骤
###### ① 拿到预训练好的模型/分类器
###### ② 冻结卷积基
###### ③ 按照实际需求随机初始化新分类器(一般是线性层)

In [8]:
model = torchvision.models.vgg16(pretrained=True) # pretrained参数指定是否使用预训练好的权重

######  拿到预训练好的模型/分类器

In [9]:
model                                             # 查看VGG-16的层结构

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

###### 冻结卷积层

In [10]:
for p in model.features.parameters():
    p.requirs_grad = False                        # 设置卷积层里面的参数不再参与梯度下降与更新

###### 按照实际需求随机初始化新分类器

In [11]:
optim     = torch.optim.Adam(model.parameters(),lr=0.0001)
loss_fn   = nn.CrossEntropyLoss()
epoches   = 20
model.classifier[-1].out_features = 4             # 将输出分类数修改为目标数量 (本例为4)
model     = model.to("cuda")

In [12]:
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

##### 定义fit函数

In [13]:
def fit(model,trainloader,testloader,epoch):
    correct = 0       # 记录正确率
    total   = 0       # 记录训练样本总数量
    running_loss = 0
    
    model.train()     # 设置为训练模式 表示DropOut会起作用
    
    for x,y in trainloader:
        x      = x.to("cuda")
        y      = y.to("cuda")
        y_pred = model(x)
        loss   = loss_fn(y_pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred,dim=1)
            correct+=(y_pred == y).sum().item()
            total  += y.size(0)
            running_loss += loss.item()
    
    epoch_loss = running_loss / len(trainloader.dataset) # 得到每一个样本在本epoch下的平均loss
    epoch_acc  = correct / total # 得到整个样本的平均预测正确率
    
    test_correct      = 0        # 记录正确率
    test_total        = 0        # 记录训练样本总数量
    test_running_loss = 0
    
    model.eval()                 # 将模型修改为预测模式 表示DropOut失效 注意在验证数据集上不需要进行Dropout
    
    with torch.no_grad():
        for x,y in trainloader:
            x      = x.to("cuda")
            y      = y.to("cuda")
            y_pred = model(x)
            loss   = loss_fn(y_pred,y)
            y_pred = torch.argmax(y_pred,dim=1)
            test_correct+=(y_pred == y).sum().item()
            test_total  += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset) # 得到每一个样本在本epoch下的平均loss
    epoch_test_acc  = test_correct / test_total                   # 得到整个样本的平均预测正确率
    
    print(
            "epoch:","\t",epoch,"\t",
            "loss: ","\t",round(epoch_loss,3),"\t",
            "accuracy: ","\t",round(epoch_acc,3),"\t",
            "test_loss: ","\t",round(epoch_test_loss,3),"\t",
            "test_accuracy: ","\t",round(epoch_test_acc,3),"\t"
        )
    return epoch_loss,epoch_acc,epoch_test_loss,epoch_test_acc

##### 对模型进行训练

In [ ]:
torch.cuda.empty_cache()
for epoch in range(epoches):
    epoch_loss,epoch_acc,epoch_test_loss,epoch_test_acc = fit(model,train_dl,test_dl,epoch)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

epoch: 	 0 	 loss:  	 0.086 	 accuracy:  	 0.833 	 test_loss:  	 0.08 	 test_accuracy:  	 0.941 	
epoch: 	 1 	 loss:  	 0.027 	 accuracy:  	 0.939 	 test_loss:  	 0.052 	 test_accuracy:  	 0.973 	
epoch: 	 2 	 loss:  	 0.022 	 accuracy:  	 0.95 	 test_loss:  	 0.027 	 test_accuracy:  	 0.988 	
epoch: 	 3 	 loss:  	 0.016 	 accuracy:  	 0.967 	 test_loss:  	 0.024 	 test_accuracy:  	 0.987 	
epoch: 	 4 	 loss:  	 0.017 	 accuracy:  	 0.963 	 test_loss:  	 0.006 	 test_accuracy:  	 0.996 	
epoch: 	 5 	 loss:  	 0.008 	 accuracy:  	 0.977 	 test_loss:  	 0.041 	 test_accuracy:  	 0.971 	
epoch: 	 6 	 loss:  	 0.002 	 accuracy:  	 0.991 	 test_loss:  	 0.003 	 test_accuracy:  	 0.999 	
epoch: 	 7 	 loss:  	 0.011 	 accuracy:  	 0.978 	 test_loss:  	 0.057 	 test_accuracy:  	 0.961 	
epoch: 	 8 	 loss:  	 0.026 	 accuracy:  	 0.95 	 test_loss:  	 0.028 	 test_accuracy:  	 0.98 	
epoch: 	 9 	 loss:  	 0.065 	 accuracy:  	 0.894 	 test_loss:  	 0.121 	 test_accuracy:  	 0.91 	
epoch: 	 10 	 l

In [ ]:
plt.plot(range(1,epoches+1),train_loss,label = "train_loss")
plt.plot(range(1,epoches+1),test_loss,label = "test_loss")
plt.legend()

In [ ]:
plt.plot(range(1, epoches+1), train_acc, label='train_acc')
plt.plot(range(1, epoches+1), test_acc, label='test_acc')
plt.legend()